In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import backend as K

import os
import numpy as np
import cv2
import progressbar
import pickle

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 90, 90
epochs = 5
batch_size = 100
n_channels = 1
rbg = 0

In [3]:
with open('/Users/alejandro/AnacondaProjects/train_set1.pkl', 'rb') as file:
    train_feat1, train_lab1 = pickle.load(file)
with open('/Users/alejandro/AnacondaProjects/dev_set1.pkl', 'rb') as file:
    dev_feat1, dev_lab1 = pickle.load(file)
with open('/Users/alejandro/AnacondaProjects/train_set2.pkl', 'rb') as file:
    train_feat2, train_lab2 = pickle.load(file)
with open('/Users/alejandro/AnacondaProjects/dev_set2.pkl', 'rb') as file:
    dev_feat2, dev_lab2 = pickle.load(file)

In [6]:
train_lab1 = np.array(train_lab1)
dev_lab1 = np.array(dev_lab1)
train_lab2 = np.array(train_lab2)
dev_lab2 = np.array(dev_lab2)

train_feat = np.vstack((train_feat1, train_feat2))
train_lab = np.concatenate((train_lab1, train_lab2))
dev_feat = np.vstack((dev_feat1, dev_feat2))
dev_lab = np.concatenate((dev_lab1, dev_lab2))

print('train features: {}, train labels: {}'.format(train_feat.shape, train_lab.shape))
print('dev features: {}, dev labels: {}'.format(dev_feat.shape, dev_lab.shape))


train features: (18000, 90, 90, 1), train labels: (18000,)
dev features: (3600, 90, 90, 1), dev labels: (3600,)


In [7]:
if K.image_data_format() == 'channels_first':
    input_shape = (n_channels, img_width, img_height)
else:
    input_shape = (img_width, img_height, n_channels)

# layer 1
model = Sequential()
model.add(Conv2D(8, (3, 3), input_shape=input_shape))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 2
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 3
model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# layer 4
model.add(Dense(128))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))

# layer 5
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

In [8]:
model.fit(
    train_feat, 
    train_lab, 
    batch_size=batch_size, 
    epochs=epochs, 
    verbose=1,
    validation_data=(dev_feat, dev_lab),
    shuffle=True)

Train on 18000 samples, validate on 3600 samples
Epoch 1/5
18000/18000 [==============================] - 152s 8ms/step - loss: 0.6144 - acc: 0.9181 - val_loss: 0.4514 - val_acc: 0.8342
Epoch 2/5
18000/18000 [==============================] - 147s 8ms/step - loss: 0.2227 - acc: 0.9602 - val_loss: 0.3776 - val_acc: 0.9547
Epoch 3/5
18000/18000 [==============================] - 142s 8ms/step - loss: 0.1736 - acc: 0.9677 - val_loss: 0.2893 - val_acc: 0.9833
Epoch 4/5
18000/18000 [==============================] - 142s 8ms/step - loss: 0.1571 - acc: 0.9693 - val_loss: 0.2465 - val_acc: 0.9842
Epoch 5/5
18000/18000 [==============================] - 145s 8ms/step - loss: 0.1402 - acc: 0.9736 - val_loss: 0.2491 - val_acc: 0.9811


In [9]:
model.save('/Users/alejandro/AnacondaProjects/cv/model2.h5')

In [10]:
with open('/Users/alejandro/AnacondaProjects/train_set.pkl', 'wb') as file:
    pickle.dump((train_feat, train_lab), file)
with open('/Users/alejandro/AnacondaProjects/dev_set.pkl', 'wb') as file:
    pickle.dump((dev_feat, dev_lab), file)

In [ ]:
# def pred_from_file(files, model, color=0):
#     predictions = []
#     with progressbar.ProgressBar(max_value=len(files)) as bar:
#         for i in range(len(files)):
#             img = cv2.imread(path + files[i], color)
#             img = img.reshape((1, img_width, img_height, n_channels))//255
#             pred = model.predict(img)
#             pred = np.argmax(pred)
#             predictions.append(pred)
#             bar.update(i)
#     return predictions

In [ ]:
# # In-sample performance
# path = '/Users/alejandro/AnacondaProjects/Hand_Data/train/hand/'
# files = os.listdir(path)
# pred_hand = pred_from_file(files, model2, color=rbg)

In [ ]:
# path = '/Users/alejandro/AnacondaProjects/Hand_Data/train/no_hand/'
# files = os.listdir(path)
# pred_no_hand = pred_from_file(files, model2, color=rbg)

In [ ]:
# FN = round(np.sum(pred_hand)/len(files), 4) * 100
# FP = 100 - round(np.sum(pred_no_hand)/len(files), 4) * 100
# TN = 100 - FN
# TP = 100 - FP
# Acc = (TP + TN) / (TP + TN + FP + FN)

# print('In-sample performance:')
# print('False negatives rate:', FN)
# print('False positive rate:', FP)
# print('Accuracy:', Acc)

In [ ]:
# # Out of sample performance
# path = '/Users/alejandro/AnacondaProjects/Hand_Data/dev/hand/'
# files = os.listdir(path)
# pred_hand = pred_from_file(files, model2, color=rbg)

In [ ]:
# path = '/Users/alejandro/AnacondaProjects/Hand_Data/dev/no_hand/'
# files = os.listdir(path)
# pred_no_hand = pred_from_file(files, model2, color=rbg)

In [ ]:
# FN = round(np.sum(pred_hand)/len(files), 4) * 100
# FP = 100 - round(np.sum(pred_no_hand)/len(files), 4) * 100
# TN = 100 - FN
# TP = 100 - FP
# Acc = (TP + TN) / (TP + TN + FP + FN)

# print('Out-of-sample performance:')
# print('False negatives rate:', FN)
# print('False positive rate:', FP)
# print('Accuracy:', Acc)